In [1]:
pip install konlpy

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/19.4 MB ? eta -:--:--
   ---------------------------------------  19.4/19.4 MB 103.0 MB/s eta 0:00:01
   ---------------------------------------- 19.4/19.4 MB 88.4 MB/s eta 0:00:00

   ---------------------------------------- 0/2 [JPype1]
   -------------------- ------------------- 1/2 [konlpy]
   -------------------- ------------------- 1/2 [konlpy]
   ---------------------------------------- 2/2 [konlpy]

Note: you may need to restart the kernel to use updated packages.


In [8]:
from selenium import webdriver 
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from openpyxl import Workbook
from konlpy.tag import Okt 
from collections import Counter
import time

options = Options()
options.add_argument("--no-sandbox") #브라우저 내에서의 코드 실행을 다른 시스템 리소스로부터 격리시켜 보안성을 높이는 기술
options.add_argument("--disable-dev-shm-usage") #크롤링한 데이터가 너무 클 경우, 브라우저가 아닌 하드디스트 내에 데이터를 넣어놨다 추후 다시 불러오는 역할
options.add_argument("--headless") #selenium이 생겼을때 브라우저 창이 뜨는걸 방지하는 것

driver = webdriver.Chrome(options=options)
url = "https://www.kurly.com/goods/1000358249"
test = driver.get(url)
time.sleep(5)

soup = BeautifulSoup(driver.page_source,"html.parser")
reviews = soup.select("p.css-y49dcn.e36z05c13")
review_lists = [review.get_text().strip() for review in reviews]

driver.quit()

wb = Workbook()
ws = wb.active
ws.title = "Kurly Reviews"
ws.append(["리뷰 내용"])

for item in review_lists:
    ws.append([item])

#한국어 형태의 분석기 사용하기
okt = Okt()
all_text = " ".join(review_lists) #리스트 값 내의 데이터를 하나의 문자열로 치환하는 방법
nouns = okt.nouns(all_text) # 문자열 데이터 내 명사 값을 추출해오기. 단, 신조어 같이 사전에 등재되어있지 않은 것은 정규식으로 패턴을 지정해 데이터를 크롤링할 수 있다
filtered_nouns = [n for n in nouns if len(n)>1]
count = Counter(filtered_nouns) 

ws_keywords = wb.create_sheet("Keyword Frequency")
ws_keywords.append(["키워드", "빈도수"])
for word, freq in count.most_common(30) :#most_common 위로부터 얼마나 많은 것까지 추출할건지
    ws_keywords.append([word, freq])
    
wb.save("kurly_reviews.xlsx")
print("엑셀 저장 완료 : kurly_reviews.xlsx")

엑셀 저장 완료 : kurly_reviews.xlsx


In [11]:
from openpyxl import load_workbook
from konlpy.tag import Okt #한국어를 자연어로 인식해서 불러올 수 있는 라이브러리와 속성
from collections import Counter #대문잘 시작된다면 프로토타입의 클래스 형식의 인스턴스 객체
wb = load_workbook("연관키워드 20250722 1448.xlsx")
ws = wb.active #희망하는 오픈 파일의 첫번째 sheet 불러오기, active는 객체가 아니니 ()는 사용하지 말것!!!

texts = [row[0] for row in ws.iter_rows(min_row = 2, max_col = 1, values_only = True ) if row[0]] 
#iter_row의 반환값은 리스트 형태로 나오며, excel내 행의 데이터를 연속적(이터러블한 객체)로 불러오는 역할
#min_row = 2, max_col = 1열 2행의 값을 불러와 값이 없는게 아니라면이라는 조건
#2행 1열부터 들어잇는 데이터들의 값만 찾아와 비어있는 값이 아니라면 리스트 형태로 값을 정렬해 text라는 변수에 넣어라

okt = Okt()
all_text = " ".join(texts)
nouns = okt.nouns(all_text) #명사만 추출해와줘

filtered_nouns = [n for n in nouns if len(n)>1] #명사의 길이가 1자 이상인 n의 요소만 리스트로 넣어줘
count = Counter(filtered_nouns)

ws_keywords = wb.create_sheet("키워드 전체분석") #지금 활성화 되어진 엑셀 파일에 새로운 sheet를 추가하는 것
ws_keywords.append(["명사", "빈도수"]) #새로 추가되는 sheet의 행에 들어갈 내용
for word, freq in sorted(count.items(), key=lambda x: x[1], reverse=True ) : #count안에 들어있는 내용을 내림차순으로 정렬해
    ws_keywords.append([word, freq])

wb.save("연간키워드_분석결과.xlsx")
print("완료 : 전체 키워드 분석 결과 저장됨")

완료 : 전체 키워드 분석 결과 저장됨


In [14]:
from openpyxl import load_workbook #엑셀관련 라이브러리에서 workbook을 불러오는 기능을 활용
from konlpy.tag import Okt #한국어를 자연어로 인식해 불러올 라이브러리 함수
from collections import Counter

wb = load_workbook("연관키워드 20250722 1548.xlsx") #엑셀파일을 불러와
ws = wb.active #불러온 엑셀파일의 sheet활성화

texts = [row[0] for row in ws.iter_rows(min_row=2, max_col=1, values_only= True) if row[0]]

okt = Okt()
all_text = " ".join(texts)
nouns = okt.nouns(all_text)

filtered_nouns = [n for n in nouns if len(n) > 1]
count = Counter(filtered_nouns)

ws_keywords = wb.create_sheet("키워드 분석")
ws_keywords.append(["명사","빈도수"])
for word, freq in sorted(count.items(), key=lambda x: x[1], reverse=True) :
    ws_keywords.append([word, freq])

wb.save("일본 여행 키워드 연간 분석 결과.xlsx")
print("저장 완료")

저장 완료


In [15]:
from openpyxl import load_workbook #엑셀관련 라이브러리에서 workbook을 불러오는 기능을 활용
from konlpy.tag import Okt #한국어를 자연어로 인식해 불러올 라이브러리 함수
from collections import Counter

wb = load_workbook("연관키워드 20250722 1559.xlsx") #엑셀파일을 불러와
ws = wb.active #불러온 엑셀파일의 sheet활성화

texts = [row[0] for row in ws.iter_rows(min_row=2, max_col=1, values_only= True) if row[0]]

okt = Okt()
all_text = " ".join(texts)
nouns = okt.nouns(all_text)

filtered_nouns = [n for n in nouns if len(n) > 1]
count = Counter(filtered_nouns)

ws_keywords = wb.create_sheet("키워드 분석")
ws_keywords.append(["명사","빈도수"])
for word, freq in sorted(count.items(), key=lambda x: x[1], reverse=True) :
    ws_keywords.append([word, freq])

wb.save("마스크팩 연간검색 키워드 정리.xlsx")
print("저장 완료")

저장 완료


In [18]:
from openpyxl import load_workbook
from konlpy.tag import Okt #한국어를 자연어로 인식해서 불러올 수 있는 라이브러리와 속성
from collections import Counter #대문잘 시작된다면 프로토타입의 클래스 형식의 인스턴스 객체
import re #정규표현식을 사용하기 위한 라이브러리

wb = load_workbook("연관키워드 20250722 1448.xlsx")
ws = wb.active #희망하는 오픈 파일의 첫번째 sheet 불러오기, active는 객체가 아니니 ()는 사용하지 말것!!!

texts = [row[0] for row in ws.iter_rows(min_row = 2, max_col = 1, values_only = True ) if row[0]] 
#iter_row의 반환값은 리스트 형태로 나오며, excel내 행의 데이터를 연속적(이터러블한 객체)로 불러오는 역할
#min_row = 2, max_col = 1열 2행의 값을 불러와 값이 없는게 아니라면이라는 조건
#2행 1열부터 들어잇는 데이터들의 값만 찾아와 비어있는 값이 아니라면 리스트 형태로 값을 정렬해 text라는 변수에 넣어라

okt = Okt()
all_text = " ".join(texts)
nouns_tokens = okt.nouns(all_text) #명사만 추출해와줘

regex_tokens = re.findall(r'[A-Za-z]*\d+[A-Za-z]+|\d+[A-Za-z]*',all_text) #명사, 숫자 상관없이 패턴에 매칭되는 모든 단어/영어 대소문자 상관없이 * 횟수 상관없이 반복되고 숫자 들어가도 상관없다
valid_one_letter = {"금","은","동"}
stopwords = {"사용","정도","조금","제품","때문","생각","만족","구매","다음"}

combined_tokens = nouns_tokens+regex_tokens
filtered_tokens = [t for t in combined_tokens if (len(t)>1 or t in valid_one_letter) and t not in stopwords]

filtered_nouns = [n for n in nouns if len(n)>1] #명사의 길이가 1자 이상인 n의 요소만 리스트로 넣어줘
count = Counter(filtered_tokens)

ws_keywords = wb.create_sheet("키워드 전체분석") #지금 활성화 되어진 엑셀 파일에 새로운 sheet를 추가하는 것
ws_keywords.append(["명사", "빈도수"]) #새로 추가되는 sheet의 행에 들어갈 내용
for word, freq in sorted(count.items(), key=lambda x: x[1], reverse=True ) : #count안에 들어있는 내용을 내림차순으로 정렬해
    ws_keywords.append([word, freq])

wb.save("연간키워드_분석결과.xlsx")
print("완료 : 전체 키워드 분석 결과 저장됨")

완료 : 전체 키워드 분석 결과 저장됨
